In [2]:
import os
from pathlib import Path
from dotenv import load_dotenv
from openai import AzureOpenAI
import argparse
import sys
import PyPDF2
import chromadb

In [3]:
# Load ENV
ENV_DIR = Path().absolute().parent / ".env"
DEV_ENV_FILE_PATH = ENV_DIR / "dev.env"
load_dotenv(DEV_ENV_FILE_PATH, override=True)

True

In [4]:

# Initialize Azure OpenAI client
client = AzureOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT_SLN"],
    api_key=os.environ["AZURE_OPENAI_API_KEY_SLN"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)

# Function to read PDF
def read_pdf(file_path):
    pdf_text = ""
    with open(file_path, "rb") as file:
        pdfReader = PyPDF2.PdfReader(file)
        for page_num in range(len(pdfReader.pages)):
            page = pdfReader.pages[page_num]
            pdf_text += page.extract_text()
    return pdf_text

In [5]:
def generate_embeddings(texts):
    response = client.embeddings.create(
        model=os.environ["AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME_ADA_SLN"],
        input=texts
    )
    return response.data[0].embedding
 

In [6]:
    DIR = Path().absolute().parent / "data"
    FILE_PATH = DIR / "war.pdf"

    # Read PDF contents
    pdf_text = read_pdf(FILE_PATH)
        
        # Remove newline characters and extra spaces
    clean_text = pdf_text.replace('\n', ' ').strip()
        
        # Split PDF text into 500-character chunks
    chunk_size = 500
    texts = [clean_text[i:i + chunk_size] for i in range(0, len(clean_text), chunk_size)] if clean_text else []

In [7]:
metadatas=[{"chunk_index": i} for i in range(len(texts))]
ids=[f"doc_{i}" for i in range(len(texts))]

In [8]:
print(metadatas)

[{'chunk_index': 0}, {'chunk_index': 1}, {'chunk_index': 2}, {'chunk_index': 3}, {'chunk_index': 4}, {'chunk_index': 5}, {'chunk_index': 6}, {'chunk_index': 7}, {'chunk_index': 8}, {'chunk_index': 9}, {'chunk_index': 10}, {'chunk_index': 11}, {'chunk_index': 12}, {'chunk_index': 13}, {'chunk_index': 14}, {'chunk_index': 15}, {'chunk_index': 16}, {'chunk_index': 17}, {'chunk_index': 18}, {'chunk_index': 19}, {'chunk_index': 20}, {'chunk_index': 21}, {'chunk_index': 22}, {'chunk_index': 23}, {'chunk_index': 24}, {'chunk_index': 25}, {'chunk_index': 26}, {'chunk_index': 27}, {'chunk_index': 28}, {'chunk_index': 29}, {'chunk_index': 30}, {'chunk_index': 31}, {'chunk_index': 32}, {'chunk_index': 33}, {'chunk_index': 34}, {'chunk_index': 35}, {'chunk_index': 36}, {'chunk_index': 37}, {'chunk_index': 38}, {'chunk_index': 39}, {'chunk_index': 40}, {'chunk_index': 41}, {'chunk_index': 42}, {'chunk_index': 43}, {'chunk_index': 44}, {'chunk_index': 45}, {'chunk_index': 46}, {'chunk_index': 47}, {

In [9]:
print(ids)

['doc_0', 'doc_1', 'doc_2', 'doc_3', 'doc_4', 'doc_5', 'doc_6', 'doc_7', 'doc_8', 'doc_9', 'doc_10', 'doc_11', 'doc_12', 'doc_13', 'doc_14', 'doc_15', 'doc_16', 'doc_17', 'doc_18', 'doc_19', 'doc_20', 'doc_21', 'doc_22', 'doc_23', 'doc_24', 'doc_25', 'doc_26', 'doc_27', 'doc_28', 'doc_29', 'doc_30', 'doc_31', 'doc_32', 'doc_33', 'doc_34', 'doc_35', 'doc_36', 'doc_37', 'doc_38', 'doc_39', 'doc_40', 'doc_41', 'doc_42', 'doc_43', 'doc_44', 'doc_45', 'doc_46', 'doc_47', 'doc_48', 'doc_49', 'doc_50', 'doc_51', 'doc_52', 'doc_53', 'doc_54', 'doc_55', 'doc_56', 'doc_57', 'doc_58', 'doc_59', 'doc_60', 'doc_61', 'doc_62', 'doc_63', 'doc_64', 'doc_65', 'doc_66', 'doc_67', 'doc_68', 'doc_69', 'doc_70', 'doc_71', 'doc_72', 'doc_73', 'doc_74', 'doc_75', 'doc_76', 'doc_77', 'doc_78', 'doc_79', 'doc_80', 'doc_81', 'doc_82', 'doc_83', 'doc_84', 'doc_85', 'doc_86', 'doc_87', 'doc_88', 'doc_89', 'doc_90', 'doc_91', 'doc_92', 'doc_93', 'doc_94', 'doc_95', 'doc_96', 'doc_97', 'doc_98', 'doc_99', 'doc_100'

In [10]:
print(texts)

['CARL VON CLAUSEWITZ  ON WAR  Edited and Translated by  MICHAEL HOWARD and PETER PARET  Introductory Essays by PETER PARET,  MICHAEL HOWARD, and BERNARD BRODIE;  with a Commentary by BERNARD BRODIE  Index by ROSALIE WEST  PRINCETON UNIVERSITY PRESS  PRINCETON, NEW JERSEY  Copyright © ${Date}. ${Publisher}. All rights reserved.CARL VON CLAUSEWITZ  Copyright © ${Date}. ${Publisher}. All rights reserved.PREPARED UNDER THE AUSPICES OF  THE CENTER OF INTERNATIONAL STUDIES  PRINCETON UNIVERSITY  A LIST', ' OF OTHER CENTER PUBLICATIONS  APPEARS AT THE BACK OF THE BOOK  Copyright © ${Date}. ${Publisher}. All rights reserved.CARL VON CLAUSEWITZ  ON WAR  Edited and Translated by  MICHAEL HOWARD and PETER PARET  Introductory Essays by PETER PARET,  MICHAEL HOWARD, and BERNARD BRODIE;  with a Commentary by BERNARD BRODIE  Index by ROSALIE WEST  PRINCETON UNIVERSITY PRESS  PRINCETON, NEW JERSEY  Copyright © ${Date}. ${Publisher}. All rights reserved.Published by Princeton University Press, 41 Will

In [11]:
embeds=[]
for i in texts:
    embeds.append(generate_embeddings(i))

In [1]:
print(len(embeds))

NameError: name 'embeds' is not defined

In [115]:
    # Create or get ChromaDB collection
chroma_client = chromadb.Client()
collection = chroma_client.get_or_create_collection(name="all-my-documents")

  
    # Add documents with embeddings to ChromaDB
collection.upsert(
        documents=texts,
        #metadatas=[{"source": str(FILE_PATH)}] * len(texts),
        metadatas=[{"chunk_index": i} for i in range(len(texts))],
        ids=[f"doc_{i}" for i in range(len(texts))],
        embeddings=embeds,
    )

In [116]:
# Function to interact with the Azure OpenAI API
def chat_with_ai(query):
    collection = chroma_client.get_or_create_collection(name="all-my-documents")
    user_embedding = generate_embeddings(query)
   
    # Search for the most similar document in ChromaDB
    results = collection.query(
        query_embeddings=[user_embedding],
        n_results=1,
        include=["documents"]
    )
   
    if results and 'documents' in results and len(results['documents']) > 0 and results['documents'][0]:
        context = results['documents'][0]
        messages = [
            {"role": "system", "content": "You are an AI assistant. Help user with the query."},
            {"role": "user", "content": query},
            {"role": "assistant", "content": f"Relevant context from document:\n\n{context}"},
        ]
        try:
            response = client.chat.completions.create(
                model=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME_SLN"],
                messages=messages,
                temperature=0.7,
                max_tokens=1000,
                top_p=1,
                frequency_penalty=0,
                presence_penalty=0,
            )
            response_content = response.choices[0].message.content
            print("Chat completion response:")
            print(response_content)
        except Exception as e:
            print(f"An error occurred: {e}")

In [117]:
def main(query=None):
    if query:
        chat_with_ai(query)
    else:
        while True:
            query = input("Enter your query (type 'exit' to quit): ")
            if query.lower() == 'exit':
                break
            chat_with_ai(query)


In [118]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Azure OpenAI CLI Chat Application")
    parser.add_argument('--query', type=str, help='User query for chat completion')
    args, unknown = parser.parse_known_args()

    main(query=args.query)

Chat completion response:
Hello! How can I assist you today?
Chat completion response:
It seems like the provided excerpt is not complete and lacks specific details about the book in question. If you could provide the title or more information about the book, I'd be happy to help with more detailed information.
Chat completion response:
I'm sorry, but I don't have access to any document you may have uploaded. However, if you describe the content or the specific information you need help with, I'd be happy to assist you!
Chat completion response:
Hello! How can I assist you today?
Chat completion response:
The provided text does not include the title of the book. It only mentions copyright information and a designation of "BOOK ONE." You might need to provide more context or check other sections of the document to find the specific title.
